In [3]:
import streamlit as st
import yfinance as yf
import pandas as pd
import quantstats as qs
import matplotlib.pyplot as plt

# Function to get stock data from Yahoo Finance
def get_stock_data(symbol, period='2y'):
    stock = yf.Ticker(symbol)
    data = stock.history(period=period)
    return data

# Function to display performance metrics
def display_metrics(data):
    st.write("### Performance Metrics")
    st.write(f"Start Date: {data.index.min().date()}")
    st.write(f"End Date: {data.index.max().date()}")
    st.write(f"Total Return: {((data['Close'][-1] / data['Close'][0]) - 1) * 100:.2f}%")
    st.write(f"Average Daily Return: {data['Close'].pct_change().mean() * 100:.2f}%")
    st.write(f"Volatility (Standard Deviation): {data['Close'].pct_change().std() * 100:.2f}%")
    st.write(f"Max Drawdown: {((data['Close'].min() / data['Close'].max()) - 1) * 100:.2f}%")

# Function to display Quantstats report
def display_quantstats_report(data, symbol):
    # QuantStats requires a returns series, so we calculate daily returns
    returns = data['Close'].pct_change().dropna()

    # Generate and display the full report in Streamlit
    st.write("### Quantitative Analysis with Quantstats")
    report_html = qs.reports.html(returns, output=None, title=f"{symbol} Performance Report")
    st.components.v1.html(report_html, height=800, scrolling=True)

# Streamlit app
st.title("Stock Data Viewer")
st.write("Fetch stock data for the last 2 years and view performance metrics.")

# Search bar for stock symbol
symbol = st.text_input("Enter a stock symbol (e.g., AAPL, TSLA, MSFT):")

if symbol:
    try:
        # Fetch stock data
        data = get_stock_data(symbol)
        
        if not data.empty:
            # Plot the stock data
            st.write(f"### Price data for {symbol}")
            st.line_chart(data['Close'])

            # Display performance metrics
            display_metrics(data)

            # Display the QuantStats report
            display_quantstats_report(data, symbol)
        else:
            st.error("No data found for the entered symbol.")
    except Exception as e:
        st.error(f"An error occurred: {e}")

